<a href="https://colab.research.google.com/github/Noha-Magdy/Text-AudioDatasets/blob/main/Librispeech/data_download_and_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading and preparing Librispeech dataset

##### needed libraries 

In [ ]:
import pandas as pd
import tarfile
import os, sys
import shutil


!pip install wget
import wget


##### Downloading and extracting Librispeech 

In [ ]:

def download_librispeech(out, splits):
"""
  Downloading librispeech dataset splits

  Arguments:
  out -- path to save the dataset on
  splits -- list of splits needed to be downloaded 

  """
  def _splits_url(split_name):
    return "https://www.openslr.org/resources/12/"+split_name+".tar.gz"

  for split in splits:
    wget.download(splits_url(split), out=out, bar="bar_thermometer")

In [ ]:
def unzip_librispeech(out, extract_path):
  """
  extracting librispeech data

  Arguments:
  out -- path of the downloaded tar files 
  extract_path -- path to extract the files on  
  """
  dirs = os.listdir( out )

  for i in dirs:
    target_name = i.split('.')
    name = out +'/'+i
    if name.endswith('.tar.gz')
      tar = tarfile.open(, "r:gz")
      tar.extractall(extract_path +'/' + target_name[0])
      tar.close()
  

##### Organize directories 

In [ ]:
def organize_dirs (extract_path, organized_path):
  dirs = os.listdir(extract_path)
  for dir in dirs:
    shutil.move(dir+ '/' + 'LibriSpeech/'+ dir , organized_path)
  
  common_files_path = extract_path + dirs[0]+'/' + "LibriSpeech"
  dirs = os.listdir( common_files_path )

  for f in dirs:
    shutil.move(common_files_path+'/'+ f , organized_path)





In [ ]:
# splits are:

dev-clean
dev-other
test-clean 
test-other
train-clean-100
train-clean-360
train-other-500


In [ ]:
# edits to be done in the data before loading it
# edit line 45 remove || 
# edit ;id to be id 
#-----------------------------------------
# use sep | and skip first 11 rows 

pd.read_csv('SPEAKERS.TXT', sep="|", skiprows=11)

,ID,SEX,SUBSET,MINUTES,NAME
0,14,F,train-clean-360,25.03,Kristin LeMoine
1,16,F,train-clean-360,25.11,Alys AtteWater
2,17,M,train-clean-360,25.04,Gord Mackenzie
3,19,F,train-clean-100,25.19,Kara Shallenberg
4,20,F,train-other-500,30.07,Gesine
...,...,...,...,...,...
2479,8975,F,train-clean-100,25.11,Daisy Flaim
2480,9000,M,train-other-500,27.26,Ramon Escamilla
2481,9022,F,train-clean-360,25.17,Claire M
2482,9023,F,train-clean-360,25.19,P. J. Morgan
